PROJECT

Set up Twitter API credentials 

In [1]:
import tweepy #https://github.com/tweepy/tweepy
from tweepy import TweepError 
import logging
import csv
import zipfile
import re
import spacy
import string
import math
import random
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 280)  #show at least 280 characters in columns, full tweets
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from string import punctuation
from collections import Counter
nltk.download('wordnet')
nltk.download('punkt') 
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.core.display import Image, display

#Twitter API credentials on seperate csv file
file = open('keys.csv')
reader = csv.reader(file)
keys_list = list(reader)[0]
consumer_key = keys_list[0]   
consumer_secret = keys_list[1]
access_key = keys_list[2]
access_secret = keys_list[3]

#authorize twitter, initialize tweepy
try:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    print("The username associated with the API credentials is:", api.auth.get_username())
except TweepError as e:
    logging.warning("There was a Tweepy error. Double check your API keys and try again.") 
    logging.warning(e)


The username associated with the API credentials is: mbuck32138


Build my data set.

In [2]:
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name, count=200, max_id=oldest)
        
        # save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")
    
    # transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]
    
    # write the csv  
    with open(f'new_{screen_name}_tweets.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    
    pass



#get_all_tweets("melbond")

In [20]:
followers = api.friends("mbuck32138")
followers_id_list = [follower.id for follower in followers]
followers_id_list

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]